# Major League Baseball Hall of Fame

The most highly coveted recognition across any sport is entrance into the Hall of Fame, it sets apart the good and great athletes from the elite. Our goal over the course of this analysis is to create a process that will analyze the collective performance of all Hall of Fame eligible players and effectively classify them as either a Hall of Fame inductee or not. Although baseball has nine different positions across the diamond, for the purpose of this analysis I will separate them into just two; hitters and pitchers. Outside of pitching and hitting statistics I will use other accolades such as All-Star apperances and awards as well as some basic fielding metrics to guide my analysis. I will break down the entire process as follows:

1. Cleaning and Organizing Data
2. Feature Engineering
3. Data Preparation
4. Initial Model
5. Advanced Modeling
6. Hyperparameter Tuning
7. Evaluation and Model Selection
8. Recommendations
9. Future Work

At the end of our process I should have two separate models that will be able to accurately classify whether batters and pitchers are members of the illustrious Hall of Fame or on the outside looking in. 

# Cleaning and Organizing Data

In [1]:
import pandas as pd
import numpy as np

In [2]:
#Import Necessary Tables
allstar = pd.read_csv('Data/AllStarFull.csv')
awards = pd.read_csv('Data/AwardsPlayers.csv')
batting = pd.read_csv('Data/Batting.csv')
fielding = pd.read_csv('Data/Fielding.csv')
hof = pd.read_csv('Data/HallOfFame.csv')
people = pd.read_csv('Data/People.csv')
pitching = pd.read_csv('Data/Pitching.csv')
pitchingpost = pd.read_csv('Data/PitchingPost.csv')
salaries = pd.read_csv('Data/Salaries.csv')

For our considerations here we want to only evaluate players who have been voted into the Hall of Fame by the Baseball Writers of America. There are other committees that can appoint someone to the hall but the BBWA is the first and foremost.

In [3]:
# Preview the Hall of Fame table.
hof.head()

,playerID,yearID,votedBy,ballots,needed,votes,inducted,category,needed_note
0,cobbty01,1936,BBWAA,226.0,170.0,222.0,Y,Player,NaN
1,ruthba01,1936,BBWAA,226.0,170.0,215.0,Y,Player,NaN
2,wagneho01,1936,BBWAA,226.0,170.0,215.0,Y,Player,NaN
3,mathech01,1936,BBWAA,226.0,170.0,205.0,Y,Player,NaN
4,johnswa01,1936,BBWAA,226.0,170.0,189.0,Y,Player,NaN


In [4]:
# As noted earlier let's filter by players and BBWA only.
hof = hof[(hof['votedBy']=='BBWAA') & (hof['category']=='Player')]

In [5]:
# Create a new table including only those inducted to the Hall of Fame.
hof_inducted = hof[hof['inducted']=='Y']

In [6]:
# Preview the new table.
hof_inducted.head()

,playerID,yearID,votedBy,ballots,needed,votes,inducted,category,needed_note
0,cobbty01,1936,BBWAA,226.0,170.0,222.0,Y,Player,NaN
1,ruthba01,1936,BBWAA,226.0,170.0,215.0,Y,Player,NaN
2,wagneho01,1936,BBWAA,226.0,170.0,215.0,Y,Player,NaN
3,mathech01,1936,BBWAA,226.0,170.0,205.0,Y,Player,NaN
4,johnswa01,1936,BBWAA,226.0,170.0,189.0,Y,Player,NaN


In [7]:
# Check for any null values.
hof_inducted.isna().sum()

playerID         0
yearID           0
votedBy          0
ballots          0
needed           0
votes            0
inducted         0
category         0
needed_note    123
dtype: int64

In [8]:
# Drop all columns besides 'playerID' and 'inducted'.
hof_inducted = hof_inducted.drop(columns=['needed_note', 'category', 'votedBy', 'yearID', 'ballots', 'needed', 'votes'], axis=1)

In [9]:
# Preview the table.
hof_inducted.head()

,playerID,inducted
0,cobbty01,Y
1,ruthba01,Y
2,wagneho01,Y
3,mathech01,Y
4,johnswa01,Y


We only care to analyze players who are eligible for Hall of Fame consideration. The two criteria are that you must have played in parts of at least ten seasons and have been retired for five seasons. Our Hall of Fame table contains data up to 2018 so we want players who's last sesason was 2012 and have played 10 plus seasons. In order to find the retired season, I am going to analyze the salary table and determine the last year in which they had a salary.

In [10]:
# Preview the salary table.
salaries.head()

,yearID,teamID,lgID,playerID,salary
0,1985,ATL,NL,barkele01,870000
1,1985,ATL,NL,bedrost01,550000
2,1985,ATL,NL,benedbr01,545000
3,1985,ATL,NL,campri01,633333
4,1985,ATL,NL,ceronri01,625000


In [11]:
# Group by 'playerID' and find the max of 'yearId' to determine a player's final year.
salaries = salaries.groupby('playerID')['yearID'].max().reset_index()

# Drop all rows where 'yearID' is later than 2012.
ineligible = salaries[salaries['yearID']>2012]
ineligible

,playerID,yearID
3,abadfe01,2016
13,abreujo02,2016
14,abreuto01,2014
16,aceveal01,2013
20,ackledu01,2016
...,...,...
5137,zimmery01,2016
5139,zitoba01,2013
5140,zobribe01,2016
5145,zuninmi01,2015


In [12]:
# Find a count of total sesaons by 'playerID' and filter to only players with 10 or more seasons.
total_seasons = batting.groupby('playerID')['yearID'].count().reset_index()
hof_eligible = total_seasons[total_seasons['yearID']>=10]

# Merge our two tables used to determine Hall of Fame eligiblity.
hof_eligible = (hof_eligible[~hof_eligible['playerID'].isin(ineligible['playerID'])])
hof_eligible

,playerID,yearID
1,aaronha01,23
3,aasedo01,13
7,abbated01,10
12,abbotgl01,12
14,abbotji01,11
...,...,...
19862,zimmehe01,14
19878,ziskri01,13
19884,zoldasa01,10
19886,zuberbi01,12


In [13]:
# Rename the last two columns to 'Retired' and 'Seasons'.
hof_eligible.columns = ['playerID', 'Seasons']

Now that we have created a table of players who have had Hall of Fame eligibility it's time to begin to clean the batting DataFrame. We want to group the data by 'playerID' so we will drop non-numeric columns, analyze null values and fill them appropriately. Although we only care about Hall of Fame eligible players for this anaylsis, I am going to keep all batters for now and separate them out later for future work.

In [14]:
# Preview the batting table.
batting.head()

,playerID,yearID,stint,teamID,lgID,G,AB,R,H,2B,...,RBI,SB,CS,BB,SO,IBB,HBP,SH,SF,GIDP
0,abercda01,1871,1,TRO,NaN,1,4,0,0,0,...,0.0,0.0,0.0,0,0.0,NaN,NaN,NaN,NaN,0.0
1,addybo01,1871,1,RC1,NaN,25,118,30,32,6,...,13.0,8.0,1.0,4,0.0,NaN,NaN,NaN,NaN,0.0
2,allisar01,1871,1,CL1,NaN,29,137,28,40,4,...,19.0,3.0,1.0,2,5.0,NaN,NaN,NaN,NaN,1.0
3,allisdo01,1871,1,WS3,NaN,27,133,28,44,10,...,27.0,1.0,1.0,0,2.0,NaN,NaN,NaN,NaN,0.0
4,ansonca01,1871,1,RC1,NaN,25,120,29,39,11,...,16.0,6.0,2.0,2,1.0,NaN,NaN,NaN,NaN,0.0


In [15]:
# Drop 'yearID', 'stint', 'teamID', and 'lgID' from the batting DataFrame
batting = batting.drop(columns=['yearID', 'stint', 'teamID', 'lgID'], axis=1)

In [16]:
# Check for null values.
batting.isna().sum()

playerID        0
G               0
AB              0
R               0
H               0
2B              0
3B              0
HR              0
RBI           756
SB           2368
CS          23541
BB              0
SO           2100
IBB         36650
HBP          2816
SH           6068
SF          36103
GIDP        25441
dtype: int64

Since this table also includes pitchers who have plate appearances and batters with very few appearances a year it is reasonable to think that these null values are equal to zero. For that reason we will fill all null values with a value of zero.

In [17]:
# Fill null values with zero.
batting = batting.fillna(0)

In [18]:
# Group by 'playerID' and find the sum totals for all other columns.
batting = batting.groupby("playerID")['G', 'AB', 'R', 'H', '2B', '3B', 'HR', 'RBI', 'SB', 'CS', 'BB', 'SO', 'HBP', 'SF'].sum().reset_index()

Lets create an aggregate of walks and hit by pitch, both of which attribute to on base percentage. Intentional walks are factored into walks already so we will not include that statistic. In addition let's calculate a couple of other key statistics; batting average, on-base percentage, slugging percentage, and OPS which is on-base percentage plus slugging percentage.

In [19]:
batting['walks'] = batting['BB'] + batting['HBP']
batting['BA'] = round(batting['H'] / batting['AB'], 3)
batting['OBP'] = round((batting['H'] + batting['walks']) / (batting['AB'] + batting['walks'] + batting['SF']), 3)
batting['1B'] = batting['H'] - batting['2B'] - batting['3B'] - batting['HR']
batting['SLUG'] = round((batting['1B'] + (batting['2B']*2) + (batting['3B']*3) + (batting['HR']*4)) / batting['AB'], 3)
batting['OPS'] = round(batting['OBP'] + batting['SLUG'], 3)

In [20]:
# Check for null values.
batting.isna().sum()

playerID       0
G              0
AB             0
R              0
H              0
2B             0
3B             0
HR             0
RBI            0
SB             0
CS             0
BB             0
SO             0
HBP            0
SF             0
walks          0
BA          2305
OBP         2257
1B             0
SLUG        2305
OPS         2305
dtype: int64

In [21]:
# The above null values are a result of not reaching base that season and will be filled with zero.
batting = batting.fillna(0)

In [22]:
# Preview cleaned batting DataFrame
batting.head()

,playerID,G,AB,R,H,2B,3B,HR,RBI,SB,...,BB,SO,HBP,SF,walks,BA,OBP,1B,SLUG,OPS
0,aardsda01,331,4,0,0,0,0,0,0.0,0.0,...,0,2.0,0.0,0.0,0.0,0.000,0.000,0,0.000,0.000
1,aaronha01,3298,12364,2174,3771,624,98,755,2297.0,240.0,...,1402,1383.0,32.0,121.0,1434.0,0.305,0.374,2294,0.555,0.929
2,aaronto01,437,944,102,216,42,6,13,94.0,9.0,...,86,145.0,0.0,6.0,86.0,0.229,0.292,155,0.327,0.619
3,aasedo01,448,5,0,0,0,0,0,0.0,0.0,...,0,3.0,0.0,0.0,0.0,0.000,0.000,0,0.000,0.000
4,abadan01,15,21,1,2,0,0,0,0.0,0.0,...,4,5.0,0.0,0.0,4.0,0.095,0.240,2,0.095,0.335


Another important Hall of Fame consideration is Allstar Game appearances. The only pieces of information we need here are 'playerID' and the sum of total appearances. 

In [23]:
# Preview allstar DataFrame.
allstar.head()

,playerID,yearID,gameNum,gameID,teamID,lgID,GP,startingPos
0,gomezle01,1933.0,0.0,ALS193307060,NYA,AL,1,1.0
1,ferreri01,1933.0,0.0,ALS193307060,BOS,AL,1,2.0
2,gehrilo01,1933.0,0.0,ALS193307060,NYA,AL,1,3.0
3,gehrich01,1933.0,0.0,ALS193307060,DET,AL,1,4.0
4,dykesji01,1933.0,0.0,ALS193307060,CHA,AL,1,5.0


In [24]:
# Drop unnecessary columns.
allstar = allstar.drop(columns=['gameNum', 'gameID', 'startingPos', 'teamID', 'lgID', 'GP'], axis=1)

In [25]:
# Check for null values.
allstar.isna().sum()

playerID    0
yearID      1
dtype: int64

In [26]:
# Find null value.
allstar[allstar['yearID'].isna()]

,playerID,yearID
5374,bailean01,NaN


In [27]:
# Preview all rows for 'playerID' equal to 'bailean01'
allstar[allstar['playerID']=='bailean01']

,playerID,yearID
4615,bailean01,2010.0
5374,bailean01,NaN


After researching 2010 allstars I have discoved that Andrew Bailey was also voted an Allstar in 2009. We will use that information to fill the null value.

In [28]:
# Fill null value.
allstar = allstar.fillna(2009.0)

In [29]:
# Check for null values.
allstar.isna().sum()

playerID    0
yearID      0
dtype: int64

In [30]:
# Group by 'playerID' to get a total count of ASG appearances.
allstar = allstar.groupby('playerID').count().reset_index()

In [31]:
# Preview cleaned allstar DataFrame
allstar.head()

,playerID,yearID
0,aaronha01,24
1,aasedo01,1
2,abreubo01,2
3,abreujo02,3
4,acunaro01,1


Award recognition is a good way to determine if a player has had a good season and many awards over a career could point to a possible Hall of Fame induction. Let's get an idea of the content of this DataFrame and determine which awards are beneficial for this analysis.

In [32]:
# Preview awards DataFrame.
awards.head()

,playerID,awardID,yearID,lgID,tie,notes
0,bondto01,Pitching Triple Crown,1877,NL,NaN,NaN
1,hinespa01,Triple Crown,1878,NL,NaN,NaN
2,heckegu01,Pitching Triple Crown,1884,AA,NaN,NaN
3,radboch01,Pitching Triple Crown,1884,NL,NaN,NaN
4,oneilti01,Triple Crown,1887,AA,NaN,NaN


In [33]:
# Find the unique values in the 'awardID' column.
awards['awardID'].unique()

array(['Pitching Triple Crown', 'Triple Crown',
       'Baseball Magazine All-Star', 'Most Valuable Player',
       'TSN All-Star', 'TSN Guide MVP',
       'TSN Major League Player of the Year', 'TSN Pitcher of the Year',
       'TSN Player of the Year', 'Rookie of the Year', 'Babe Ruth Award',
       'Lou Gehrig Memorial Award', 'World Series MVP', 'Cy Young Award',
       'Gold Glove', 'TSN Fireman of the Year', 'All-Star Game MVP',
       'Hutch Award', 'Roberto Clemente Award',
       'Rolaids Relief Man Award', 'NLCS MVP', 'ALCS MVP',
       'Silver Slugger', 'Branch Rickey Award', 'Hank Aaron Award',
       'TSN Reliever of the Year', 'Comeback Player of the Year',
       'Outstanding DH Award', 'Reliever of the Year Award'], dtype=object)

The traditional All-Star game first came to be during the 1933 season. However, as we can see above there is an award listed as the Baseball Magazine All-Star which originated in 1914. In this next section I am going to get a count of that award by player before the 1933 season and append it to our original All-Star game list.

In [34]:
# Drop unnecessary columns ('lgID', 'tie', 'notes').
awards = awards.drop(columns=['lgID', 'tie', 'notes'], axis=1)

In [35]:
# Check for null values
awards.isna().sum()

playerID    0
awardID     0
yearID      0
dtype: int64

In [36]:
# Preview revised table.
awards.head()

,playerID,awardID,yearID
0,bondto01,Pitching Triple Crown,1877
1,hinespa01,Triple Crown,1878
2,heckegu01,Pitching Triple Crown,1884
3,radboch01,Pitching Triple Crown,1884
4,oneilti01,Triple Crown,1887


In [37]:
# Filter to only years before 1933
awards2 = awards[awards['yearID']<1933]

# It was common to play both pitcher and another position in the early 1900's. Drop duplicates in case voted All-Star for both.
awards2 = awards2.drop_duplicates()

# Set 'awardID' equal to 'Baseball Magazine All-Star'.
allstar2 = awards2[awards2['awardID']=='Baseball Magazine All-Star']

# Preview the new All-Star list.
allstar2.head()

,playerID,awardID,yearID
13,chaseha01,Baseball Magazine All-Star,1908
14,lajoina01,Baseball Magazine All-Star,1908
15,lordha01,Baseball Magazine All-Star,1908
16,crigelo01,Baseball Magazine All-Star,1908
17,sullibi03,Baseball Magazine All-Star,1908


In [38]:
# Drop unnecessary columns and group by 'playerID' to find the count.
allstar2 = allstar2.drop('awardID', axis=1)
allstar2 = allstar2.groupby('playerID').count()
allstar2.head()

,yearID
playerID,
adamsba01,1
ainsmed01,1
alexape01,10
archeji01,5
averiea01,3


In [39]:
# Append the new All-Star list to the original.
allstar_final = pd.concat([allstar, allstar2], sort=True)

In [40]:
# Group together to finalize new All-Star DataFrame and rename columns.
allstar_final = allstar_final.groupby("playerID")['yearID'].sum().reset_index()
allstar_final.columns = ['playerID', 'Apps']
allstar_final.head()

,playerID,Apps
0,aaronha01,24
1,aasedo01,1
2,abreubo01,2
3,abreujo02,3
4,acunaro01,1


Moving on to the awards DataFrame, I see from above that there are several awards that do not represent on-field performance. For the purposes of this analysis we only want awards related to on-field performance. We will select those awards and create a unique column for each and develop a count by player.

In [41]:
# Select awards related to on-field performance.
awards = awards[awards.awardID.isin(['Most Valuable Player', 'Rolaids Relief Man Award', 'Cy Young Award',
                                     'Gold Glove', 'Silver Slugger', 'Rookie of the Year', 'Hank Aaron Award'])]

In [42]:
# Preview DataFrame.
awards.head()

,playerID,awardID,yearID
188,cobbty01,Most Valuable Player,1911
189,schulfr01,Most Valuable Player,1911
226,speaktr01,Most Valuable Player,1912
227,doylela01,Most Valuable Player,1912
258,johnswa01,Most Valuable Player,1913


In [43]:
# Pivot award DataFrame to show each award as a unique column.
awards_df = pd.pivot_table(awards, index=['playerID'],
                    columns=['awardID'], aggfunc='count')
awards_df.head()

yearID                                                   \
awardID   Cy Young Award Gold Glove Hank Aaron Award Most Valuable Player   
playerID                                                                    
aaronha01            NaN        3.0              NaN                  1.0   
abreubo01            NaN        1.0              NaN                  NaN   
abreujo02            NaN        NaN              NaN                  NaN   
ageeto01             NaN        2.0              NaN                  NaN   
alfonan01            NaN        NaN              NaN                  NaN   

                                                                      
awardID   Rolaids Relief Man Award Rookie of the Year Silver Slugger  
playerID                                                              
aaronha01                      NaN                NaN            NaN  
abreubo01                      NaN                NaN            1.0  
abreujo02                      NaN                1.0            1.0  
ageeto01                       NaN                1.0            NaN  
alfonan01                      1.0                NaN            NaN

In [44]:
# Convert pivot table back to a DataFrame and fill null values to zero.
awards_df1 = pd.DataFrame(awards_df.to_records())
awards_df1.columns = ['playerID', 'Cy Young', 'Gold Glove', 'Hank Aaron', 'Most Valuable Player', 'Rolaids Relief Man Award', 'Rookie of the Year', 'Silver Slugger']
awards_df1 = awards_df1.fillna(0)
awards_df1.head()

,playerID,Cy Young,Gold Glove,Hank Aaron,Most Valuable Player,Rolaids Relief Man Award,Rookie of the Year,Silver Slugger
0,aaronha01,0.0,3.0,0.0,1.0,0.0,0.0,0.0
1,abreubo01,0.0,1.0,0.0,0.0,0.0,0.0,1.0
2,abreujo02,0.0,0.0,0.0,0.0,0.0,1.0,1.0
3,ageeto01,0.0,2.0,0.0,0.0,0.0,1.0,0.0
4,alfonan01,0.0,0.0,0.0,0.0,1.0,0.0,0.0


When considering indivduals who have had Hall of Fame careers, hitting and pitching are typically the two most important aspects. However, we do not want to ignore contributions made in the field as well. We'll include some of the key fielding metrics in the DataFrame above to include in our analysis.

In [45]:
# Preview fielding DataFrame.
fielding.head()

,playerID,yearID,stint,teamID,lgID,POS,G,GS,InnOuts,PO,A,E,DP,PB,WP,SB,CS,ZR
0,abercda01,1871,1,TRO,NaN,SS,1,1.0,24.0,1,3,2.0,0,NaN,NaN,NaN,NaN,NaN
1,addybo01,1871,1,RC1,NaN,2B,22,22.0,606.0,67,72,42.0,5,NaN,NaN,NaN,NaN,NaN
2,addybo01,1871,1,RC1,NaN,SS,3,3.0,96.0,8,14,7.0,0,NaN,NaN,NaN,NaN,NaN
3,allisar01,1871,1,CL1,NaN,2B,2,0.0,18.0,1,4,0.0,0,NaN,NaN,NaN,NaN,NaN
4,allisar01,1871,1,CL1,NaN,OF,29,29.0,729.0,51,3,7.0,1,NaN,NaN,NaN,NaN,NaN


In [46]:
# Drop unnecessary columns.
fielding = fielding.drop(columns=['ZR', 'yearID', 'stint', 'teamID', 'lgID', 'GS', 
                                  'InnOuts', 'DP', 'WP', 'PB', 'SB', 'CS'], axis=1)

In [47]:
# Check for null values.
fielding.isna().sum()

playerID    0
POS         0
G           0
PO          0
A           0
E           1
dtype: int64

In [48]:
# Fill null value with zero.
fielding = fielding.fillna(0)

It's not uncommon for someone to play second base one season and shortstop the next. In order to group data properly we are going to assign any position player as "B" for batter and any pitcher as "P" for pitcher. We will create a function that will look at the position and do just that. Some players will appear twice if they have played both a position in the field and pitcher.

In [49]:
# Create a function that assigns all position players as B and pitchers as P.
def f(row):
    if row['POS'] == 'P':
        val = 'P'
    else:
        val = 'B'
    return val

In [50]:
# Apply the function above to the fielding DataFrame.
fielding['Position'] = fielding.apply(f, axis=1)

In [51]:
# Group by 'playerID' and 'Position' finding the sum of the remaining key metrics.
fielding = fielding.groupby(['playerID', 'Position'])['PO', 'A', 'E'].sum().reset_index()

# Create new metric, fielding percentage, using putouts, assists and errors.
fielding['FP'] = round((fielding['PO'] + fielding['A']) / (fielding['PO'] + fielding['A'] + fielding['E']), 2)

# Fill null values as zero for the newly created fielding percentage metric.
fielding = fielding.fillna(0)

In [52]:
# Rename columns and preview final version of table.
fielding.columns = ['playerID', 'Position', 'PO', 'A', 'E', 'FP']
fielding.head()

,playerID,Position,PO,A,E,FP
0,aardsda01,P,11,29,3.0,0.93
1,aaronha01,B,7436,429,144.0,0.98
2,aaronto01,B,1317,113,22.0,0.98
3,aasedo01,P,67,135,13.0,0.94
4,abadan01,B,37,1,1.0,0.97


Finally, similar to the batting DataFrame from above I am going to create a new DataFrame grouped by 'playerID' and add a few new metrics as well. Once again, I am not going to limit this table to only Hall of Fame eligible players during this step to make possible future analysis more easily accessible. 

In [53]:
# Preview pitching DataFrame.
pitching.head()

,playerID,yearID,stint,teamID,lgID,W,L,G,GS,CG,...,IBB,WP,HBP,BK,BFP,GF,R,SH,SF,GIDP
0,bechtge01,1871,1,PH1,NaN,1,2,3,3,2,...,NaN,7,NaN,0,146.0,0,42,NaN,NaN,NaN
1,brainas01,1871,1,WS3,NaN,12,15,30,30,30,...,NaN,7,NaN,0,1291.0,0,292,NaN,NaN,NaN
2,fergubo01,1871,1,NY2,NaN,0,0,1,0,0,...,NaN,2,NaN,0,14.0,0,9,NaN,NaN,NaN
3,fishech01,1871,1,RC1,NaN,4,16,24,24,22,...,NaN,20,NaN,0,1080.0,1,257,NaN,NaN,NaN
4,fleetfr01,1871,1,NY2,NaN,0,1,1,1,1,...,NaN,0,NaN,0,57.0,0,21,NaN,NaN,NaN


In [54]:
# Find null values if any.
pitching.isna().sum()

playerID        0
yearID          0
stint           0
teamID          0
lgID          132
W               0
L               0
G               0
GS              0
CG              0
SHO             0
SV              0
IPouts          0
H               0
ER              0
HR              0
BB              0
SO              0
BAOpp        4441
ERA            94
IBB         14578
WP              0
HBP           734
BK              0
BFP             3
GF              0
R               0
SH          19187
SF          19187
GIDP        20318
dtype: int64

In [55]:
pitching.fillna(0)

,playerID,yearID,stint,teamID,lgID,W,L,G,GS,CG,...,IBB,WP,HBP,BK,BFP,GF,R,SH,SF,GIDP
0,bechtge01,1871,1,PH1,0,1,2,3,3,2,...,0.0,7,0.0,0,146.0,0,42,0.0,0.0,0.0
1,brainas01,1871,1,WS3,0,12,15,30,30,30,...,0.0,7,0.0,0,1291.0,0,292,0.0,0.0,0.0
2,fergubo01,1871,1,NY2,0,0,0,1,0,0,...,0.0,2,0.0,0,14.0,0,9,0.0,0.0,0.0
3,fishech01,1871,1,RC1,0,4,16,24,24,22,...,0.0,20,0.0,0,1080.0,1,257,0.0,0.0,0.0
4,fleetfr01,1871,1,NY2,0,0,1,1,1,1,...,0.0,0,0.0,0,57.0,0,21,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48394,zeuchtj01,2020,1,TOR,AL,1,0,3,1,0,...,0.0,0,0.0,0,47.0,1,2,0.0,0.0,0.0
48395,zimmebr02,2020,1,BAL,AL,0,0,2,1,0,...,0.0,2,2.0,0,31.0,1,6,0.0,0.0,1.0
48396,zimmejo02,2020,1,DET,AL,0,0,3,2,0,...,0.0,0,0.0,0,28.0,0,6,0.0,0.0,1.0
48397,zimmeky01,2020,1,KCA,AL,1,0,16,1,0,...,0.0,2,1.0,0,91.0,4,4,0.0,0.0,3.0


In [56]:
pitching.dtypes

playerID     object
yearID        int64
stint         int64
teamID       object
lgID         object
W             int64
L             int64
G             int64
GS            int64
CG            int64
SHO           int64
SV            int64
IPouts        int64
H             int64
ER            int64
HR            int64
BB            int64
SO            int64
BAOpp       float64
ERA         float64
IBB         float64
WP            int64
HBP         float64
BK            int64
BFP         float64
GF            int64
R             int64
SH          float64
SF          float64
GIDP        float64
dtype: object

We are going to create a few new metrics which are all very important for analysis. The following is an explanation of each metric:

ERA: Earned Run Average is the average number of runs a pitcher gives up per 9 innings.

WHIP: Walks and Hits per Inning Pitched as the name suggests is an average of baserunners allowed each inning via walk or hit.

K/9: Strikeouts per 9 innings pitched is the average number of strikeouts over 9 innings.

K/BB: Strikeouts to Walk ratio shows the number of strikeouts compared to walks, a good measuring stick of a pitcher's control.

In [57]:
# First group by 'playerID' and find the aggregate sum of the accompanying metics.
pitching = pitching.groupby("playerID")['W', 'L', 'G', 'GS', 'CG', 'SHO', 'SV', 'IPouts', 'SF',
                                       'H', 'ER', 'HR', 'BB', 'SO', 'R', 'BFP', 'HBP', 'SH'].sum().reset_index()

# Create the new metrics mentioned above.
pitching['IP'] = round(pitching['IPouts'] / 3, 1)
pitching['ERA'] = round(pitching['ER'] / pitching['IP'] * 9, 2)
pitching['WHIP'] = round((pitching['BB'] + pitching['H']) / pitching['IP'], 2)
pitching['K/9'] = round(pitching['SO'] / pitching['IP'] * 9, 2)
pitching['K/BB'] = round(pitching['SO'] / pitching['BB'], 2)
pitching['BAA'] = round(pitching['H'] / (pitching['BFP'] - pitching['HBP'] - pitching['SH'] - pitching['SF'] - pitching['BB']), 3)

# Fill the new null values with zero.
pitching = pitching.fillna(0)

#Preview the finalize pitching DataFrame.
pitching.head()

,playerID,W,L,G,GS,CG,SHO,SV,IPouts,SF,...,R,BFP,HBP,SH,IP,ERA,WHIP,K/9,K/BB,BAA
0,aardsda01,16,18,331,0,0,0,69,1011,11.0,...,169,1475.0,16.0,17.0,337.0,4.27,1.42,9.08,1.86,0.237
1,aasedo01,66,60,448,91,22,5,82,3328,34.0,...,503,4730.0,7.0,50.0,1109.3,3.80,1.39,5.20,1.40,0.259
2,abadfe01,8,29,384,6,0,0,2,992,12.0,...,143,1399.0,12.0,7.0,330.7,3.67,1.29,7.62,2.41,0.247
3,abbeybe01,22,40,79,65,52,0,1,1704,0.0,...,442,2568.0,26.0,0.0,568.0,4.52,1.55,2.55,0.84,0.292
4,abbeych01,0,0,1,0,0,0,0,6,0.0,...,3,12.0,0.0,0.0,2.0,4.50,3.00,0.00,0.00,0.500


In [58]:
pitching = pitching.drop(columns=['SF', 'BFP', 'HBP', 'SH'], axis=1)

The last piece of information that I want to include in this DataFrame is the full name of each player. That can be found in the people dataset above. We are going to concatenate the first name and last name columns to create a full name column tied to each playerID.

In [59]:
# Preview the people DataFrame.
people.head()

,playerID,birthYear,birthMonth,birthDay,birthCountry,birthState,birthCity,deathYear,deathMonth,deathDay,...,nameLast,nameGiven,weight,height,bats,throws,debut,finalGame,retroID,bbrefID
0,aardsda01,1981.0,12.0,27.0,USA,CO,Denver,NaN,NaN,NaN,...,Aardsma,David Allan,215.0,75.0,R,R,2004-04-06,2015-08-23,aardd001,aardsda01
1,aaronha01,1934.0,2.0,5.0,USA,AL,Mobile,2021.0,1.0,22.0,...,Aaron,Henry Louis,180.0,72.0,R,R,1954-04-13,1976-10-03,aaroh101,aaronha01
2,aaronto01,1939.0,8.0,5.0,USA,AL,Mobile,1984.0,8.0,16.0,...,Aaron,Tommie Lee,190.0,75.0,R,R,1962-04-10,1971-09-26,aarot101,aaronto01
3,aasedo01,1954.0,9.0,8.0,USA,CA,Orange,NaN,NaN,NaN,...,Aase,Donald William,190.0,75.0,R,R,1977-07-26,1990-10-03,aased001,aasedo01
4,abadan01,1972.0,8.0,25.0,USA,FL,Palm Beach,NaN,NaN,NaN,...,Abad,Fausto Andres,184.0,73.0,L,L,2001-09-10,2006-04-13,abada001,abadan01


In [60]:
# Concatenate the 'nameFirst' and 'nameLast' columns.
people['fullName'] = people['nameFirst'] + ' ' + people['nameLast']

In [61]:
# Create a new people table that contains only 'playerID' and 'fullName'.
people = people[['playerID', 'fullName']]

In [62]:
# Preview finalized people DataFrame
people.head()

,playerID,fullName
0,aardsda01,David Aardsma
1,aaronha01,Hank Aaron
2,aaronto01,Tommie Aaron
3,aasedo01,Don Aase
4,abadan01,Andy Abad


Now that I have all of the tables cleaned and restructured to only include the necessary columns I am going to merge them all together to create four new tables. The first will contain all records from the batters table merged with our new fielding, ASG, awards, and people table. The second will be the same however, will only include hall of fame eligible batters. The other two tables will be structured in the same manner but for pitchers. 

In [63]:
# Merge the people and batting DFs on 'playerID' to include all batting rows. 
batters = pd.merge(people, batting, how='right', on='playerID')

# Merge the batters and fielding DFs on 'playerID' to include all batters rows. 
batters1 = pd.merge(batters, fielding, how='left', on='playerID')

# Merge the batters1 and awards_df1 DFs on 'playerID' to include all batters1 rows. 
batters2 = pd.merge(batters1, awards_df1, how='left', on='playerID')

# Merge the batters2 and allstar_final DFs on 'playerID' to include all batters2 rows. 
batters_df = pd.merge(batters2, allstar_final, how='left', on='playerID')

In [64]:
# Drop duplicates if any.
batters_df.drop_duplicates()

,playerID,fullName,G,AB,R,H,2B,3B,HR,RBI,...,E,FP,Cy Young,Gold Glove,Hank Aaron,Most Valuable Player,Rolaids Relief Man Award,Rookie of the Year,Silver Slugger,Apps
0,aardsda01,David Aardsma,331,4,0,0,0,0,0,0.0,...,3.0,0.93,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,aaronha01,Hank Aaron,3298,12364,2174,3771,624,98,755,2297.0,...,144.0,0.98,0.0,3.0,0.0,1.0,0.0,0.0,0.0,24.0
2,aaronto01,Tommie Aaron,437,944,102,216,42,6,13,94.0,...,22.0,0.98,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,aasedo01,Don Aase,448,5,0,0,0,0,0,0.0,...,13.0,0.94,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
4,abadan01,Andy Abad,15,21,1,2,0,0,0,0.0,...,1.0,0.97,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21303,zupofr01,Frank Zupo,16,18,3,3,1,0,0,0.0,...,2.0,0.94,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
21304,zuvelpa01,Paul Zuvella,209,491,41,109,17,2,2,20.0,...,23.0,0.97,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
21305,zuverge01,George Zuverink,266,142,5,21,2,1,0,7.0,...,7.0,0.96,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
21306,zwilldu01,Dutch Zwilling,366,1280,167,364,76,15,30,202.0,...,25.0,0.97,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [65]:
# Rename 'Apps' to 'ASG' and preview DataFrame.
batters_df.rename(columns = {"Apps": "ASG"},  
          inplace = True)

batters_df.head()

,playerID,fullName,G,AB,R,H,2B,3B,HR,RBI,...,E,FP,Cy Young,Gold Glove,Hank Aaron,Most Valuable Player,Rolaids Relief Man Award,Rookie of the Year,Silver Slugger,ASG
0,aardsda01,David Aardsma,331,4,0,0,0,0,0,0.0,...,3.0,0.93,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,aaronha01,Hank Aaron,3298,12364,2174,3771,624,98,755,2297.0,...,144.0,0.98,0.0,3.0,0.0,1.0,0.0,0.0,0.0,24.0
2,aaronto01,Tommie Aaron,437,944,102,216,42,6,13,94.0,...,22.0,0.98,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,aasedo01,Don Aase,448,5,0,0,0,0,0,0.0,...,13.0,0.94,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
4,abadan01,Andy Abad,15,21,1,2,0,0,0,0.0,...,1.0,0.97,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [66]:
# Merge the HOF eligible table with the batters_df on 'playerID'.
eligible = pd.merge(hof_eligible, batters_df, how='left', on='playerID')

# The updated table with the eligible batters with the hof_inducted players.
eligible_df = pd.merge(eligible, hof_inducted, how='left', on='playerID')

In [67]:
eligible_df.isna().sum()

playerID                       0
Seasons                        0
fullName                       0
G                              0
AB                             0
R                              0
H                              0
2B                             0
3B                             0
HR                             0
RBI                            0
SB                             0
CS                             0
BB                             0
SO                             0
HBP                            0
SF                             0
walks                          0
BA                             0
OBP                            0
1B                             0
SLUG                           0
OPS                            0
Position                       0
PO                             0
A                              0
E                              0
FP                             0
Cy Young                    3374
Gold Glove                  3374
Hank Aaron

In [68]:
eligible_df['inducted'] = eligible_df[['inducted']].fillna('N')
eligible_df = eligible_df.fillna(0)
eligible_df = eligible_df[(eligible_df['Position']=='B') & (eligible_df['AB']>2500)]
eligible_df

,playerID,Seasons,fullName,G,AB,R,H,2B,3B,HR,...,FP,Cy Young,Gold Glove,Hank Aaron,Most Valuable Player,Rolaids Relief Man Award,Rookie of the Year,Silver Slugger,ASG,inducted
0,aaronha01,23,Hank Aaron,3298,12364,2174,3771,624,98,755,...,0.98,0.0,3.0,0.0,1.0,0.0,0.0,0.0,24.0,Y
2,abbated01,10,Ed Abbaticchio,855,3044,355,772,99,43,11,...,0.93,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,N
9,abreubo01,20,Bobby Abreu,2425,8480,1453,2470,574,59,288,...,0.98,0.0,1.0,0.0,0.0,0.0,0.0,1.0,2.0,N
13,adairje01,15,Jerry Adair,1165,4019,378,1022,163,19,57,...,0.98,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,N
15,adamsbo03,15,Bobby Adams,1281,4019,591,1082,188,49,37,...,0.96,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,N
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3927,zernigu01,12,Gus Zernial,1234,4131,572,1093,159,22,237,...,0.97,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,N
3928,zimmech01,20,Chief Zimmer,1280,4546,617,1225,222,76,26,...,0.95,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,N
3929,zimmedo01,14,Don Zimmer,1095,3283,353,773,130,22,91,...,0.96,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,N
3930,zimmehe01,14,Heinie Zimmerman,1456,5304,695,1566,275,105,58,...,0.93,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,N


In [69]:
pitchers = pd.merge(people, pitching, how='right', on='playerID')
pitchers1 = pd.merge(pitchers, fielding, how='left', on='playerID')
pitchers2 = pd.merge(pitchers1, awards_df1, how='left', on='playerID')
pitchers_df = pd.merge(pitchers2, allstar_final, how='left', on='playerID')

In [70]:
pitchers_df.drop_duplicates()

,playerID,fullName,W,L,G,GS,CG,SHO,SV,IPouts,...,E,FP,Cy Young,Gold Glove,Hank Aaron,Most Valuable Player,Rolaids Relief Man Award,Rookie of the Year,Silver Slugger,Apps
0,aardsda01,David Aardsma,16,18,331,0,0,0,69,1011,...,3.0,0.93,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,aasedo01,Don Aase,66,60,448,91,22,5,82,3328,...,13.0,0.94,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
2,abadfe01,Fernando Abad,8,29,384,6,0,0,2,992,...,2.0,0.96,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,abbeybe01,Bert Abbey,22,40,79,65,52,0,1,1704,...,22.0,0.87,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,abbeych01,Charlie Abbey,0,0,1,0,0,0,0,6,...,100.0,0.91,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11398,zuberbi01,Bill Zuber,43,42,224,65,23,3,6,2358,...,5.0,0.96,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11399,zuberty01,Tyler Zuber,1,2,23,0,0,0,0,66,...,0.0,1.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11400,zumayjo01,Joel Zumaya,13,12,171,0,0,0,5,629,...,2.0,0.91,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11401,zuverge01,George Zuverink,32,36,265,31,9,2,40,1927,...,7.0,0.96,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [71]:
pitchers_df.rename(columns = {"Apps": "ASG"},  
          inplace = True)

pitchers_df.head()

,playerID,fullName,W,L,G,GS,CG,SHO,SV,IPouts,...,E,FP,Cy Young,Gold Glove,Hank Aaron,Most Valuable Player,Rolaids Relief Man Award,Rookie of the Year,Silver Slugger,ASG
0,aardsda01,David Aardsma,16,18,331,0,0,0,69,1011,...,3.0,0.93,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,aasedo01,Don Aase,66,60,448,91,22,5,82,3328,...,13.0,0.94,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
2,abadfe01,Fernando Abad,8,29,384,6,0,0,2,992,...,2.0,0.96,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,abbeybe01,Bert Abbey,22,40,79,65,52,0,1,1704,...,22.0,0.87,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,abbeych01,Charlie Abbey,0,0,1,0,0,0,0,6,...,100.0,0.91,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [72]:
eligible1 = pd.merge(hof_eligible, pitchers_df, on='playerID')
eligible_df1 = pd.merge(eligible1, hof_inducted, how='left', on='playerID')
eligible_df1.isna().sum()

playerID                       0
Seasons                        0
fullName                       0
W                              0
L                              0
G                              0
GS                             0
CG                             0
SHO                            0
SV                             0
IPouts                         0
H                              0
ER                             0
HR                             0
BB                             0
SO                             0
R                              0
IP                             0
ERA                            0
WHIP                           0
K/9                            0
K/BB                           0
BAA                            0
Position                       0
PO                             0
A                              0
E                              0
FP                             0
Cy Young                    1830
Gold Glove                  1830
Hank Aaron

In [73]:
eligible_df1['inducted'] = eligible_df1[['inducted']].fillna('N')
eligible_df1 = eligible_df1.fillna(0)
eligible_df1 = eligible_df1[(eligible_df1['Position']=='P') & (eligible_df1['G']>=50)]
eligible_df1

,playerID,Seasons,fullName,W,L,G,GS,CG,SHO,SV,...,FP,Cy Young,Gold Glove,Hank Aaron,Most Valuable Player,Rolaids Relief Man Award,Rookie of the Year,Silver Slugger,ASG,inducted
0,aasedo01,13,Don Aase,66,60,448,91,22,5,82,...,0.94,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,N
1,abbotgl01,12,Glenn Abbott,62,83,248,206,37,5,0,...,0.95,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,N
2,abbotji01,11,Jim Abbott,87,108,263,254,31,6,0,...,0.98,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,N
3,abbotpa01,12,Paul Abbott,43,37,162,112,1,0,0,...,0.96,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,N
4,abernte02,17,Ted Abernathy,63,69,681,34,7,2,148,...,0.95,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,N
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2009,zahnge01,14,Geoff Zahn,111,109,304,270,79,20,1,...,0.95,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,N
2010,zambrca01,12,Carlos Zambrano,132,91,354,302,10,5,0,...,0.95,0.0,0.0,0.0,0.0,0.0,0.0,3.0,3.0,N
2013,zoldasa01,10,Sam Zoldak,43,53,250,93,30,5,8,...,0.98,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,N
2014,zuberbi01,12,Bill Zuber,43,42,224,65,23,3,6,...,0.96,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,N


In [74]:
# Create batters_df as a csv file.
batters_df.to_csv('batters_df.csv',  index=False)

In [75]:
# Create pitchers_df as a csv file.
pitchers_df.to_csv('pitchers_df.csv',  index=False)

In [76]:
# Create eligible_df as a csv file.
eligible_df.to_csv('eligible_batters.csv', index=False)

In [77]:
# Create eligible_df1 as a csv file.
eligible_df1.to_csv('eligible_pitchers.csv', index=False)